# Project demo

## Imports

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import json
import torch
from sktime.distances import distance_factory
import pytorch_lightning as pl

In [ ]:
import os
import sys
sys.path.append(os.path.join('..', 'src'))

from data_loaders.melbourne_pedestrian import MelbounePedestrianDataModule
from trainers.generation_trainer import TSGenerationModule
import utils.utils_functions as ufn
from utils.time_series_functions import *

## Config

In [ ]:
NOISE_DIM = 100
config = {   
    "dataset_name": "melbourne_pedestrian",
    "dataset_config": {},
    "loader_config": {
                        "batch_size": 32,
                        "num_workers": 20,
                        "pin_memory": True
                    }
}
trainer_args = {
        "max_steps": 20000,
        "enable_checkpointing": True,
        "default_root_dir": f"out",
        "accelerator": "auto",#"cuda",
}

output_data_dir = os.path.join("..", "data", "melbourne_pedestrian")
model_configs_path = os.path.join("..", "configs", "models")
gen_configs_path = os.path.join(model_configs_path, "generator_InceptionTime.json")
disc_configs_path = os.path.join(model_configs_path, "InceptionTime.json")

gen_ckpt_load_dir = os.path.join("..", "out", "generator0", "version_7", "ckpts")
gan_ckpt_load_dir = os.path.join("..", "out", "gan0", "version_2", "ckpts")

## Creating resources

In [ ]:
if not os.path.exists(output_data_dir):
    os.makedirs(output_data_dir)

data_module = MelbounePedestrianDataModule(config["dataset_config"], loader_config=config["loader_config"])
dataset = data_module.train_dataset
labels = dataset.unique_labels
data_loader = data_module.train_dataloader()
it = iter(data_loader)

In [ ]:
gen_config = ufn.parse_config(gen_configs_path)
disc_config = ufn.parse_config(disc_configs_path)

generator = ufn.load_generator(gen_config, data_module)
discriminator = ufn.load_discriminator(disc_config, data_module)

In [ ]:
generation_module = TSGenerationModule(
    model=generator,
    optimizer_config=gen_config["optimizer_params"],
    num_classes=data_module.num_classes,
    noise_dim=gen_config["noise_dim"],
    l2_lambda=0.5,
    l1_lambda=0.5,
)

trainer_args = trainer_args
trainer = pl.Trainer(**trainer_args)

In [ ]:
tuner = pl.tuner.Tuner(trainer)
results = tuner.lr_find(
    generation_module, 
    # optimizer = optimizer,
    attr_name="lr",
    train_dataloaders=data_module, 
    min_lr=10e-8,
    max_lr=10.0,
)
# Plot with
fig = results.plot(suggest=True)
fig.show()